**Import the necessary libraries**

In [1]:
!pip install transformers bitsandbytes datasets accelerate peft trl wandb evaluate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 31.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 100.8 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s 

In [2]:
import torch
from transformers import AutoTokenizer , AutoModelForCausalLM , BitsAndBytesConfig
from datasets import load_dataset

In [3]:
from peft import get_peft_model, LoraConfig, TaskType

2025-08-21 03:54:49.920821: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755748490.150774      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755748490.200840      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
!nvidia-smi

Thu Aug 21 03:56:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P8             10W /   70W |       3MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

**Load the TinyLlama model**

In [5]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

**Initialize the tokenizer**

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name , use_fast = True)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

**For Quantiziting the model creat the config model**

In [7]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,           
    bnb_4bit_quant_type="nf4",    
    bnb_4bit_use_double_quant=True, 
    bnb_4bit_compute_dtype=torch.bfloat16, 
)

**Quantiziting the model with the defined configurations**

In [63]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map = "auto",
    trust_remote_code=True,
       
)

In [9]:
dataset = load_dataset("theatticusproject/cuad-qa")

README.md: 0.00B [00:00, ?B/s]

cuad-qa.py: 0.00B [00:00, ?B/s]

The repository for theatticusproject/cuad-qa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/theatticusproject/cuad-qa.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split:   0%|          | 0/22450 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4182 [00:00<?, ? examples/s]

In [10]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 22450
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 4182
    })
})


In [11]:
def qa_formatter(example):
    # Sometimes "answers" is a dict of lists; get the first answer
    answer = example['answers']['text'][0] if example['answers']['text'] else ""
    return {
        "prompt": f"### Question:\n{example['question']}\n\n### Context:\n{example['context']}\n\n### Answer:\n{answer}"
    }

In [12]:
formatted_dataset = {
    split: ds.map(qa_formatter)
    for split, ds in dataset.items()
}

Map:   0%|          | 0/22450 [00:00<?, ? examples/s]

Map:   0%|          | 0/4182 [00:00<?, ? examples/s]

In [13]:
def tokenizer_fun(example):
    out = tokenizer(
        example["prompt"],
        padding="max_length",
        truncation=True,
        max_length=256 ,
    )
    out["labels"] = out["input_ids"]
    return out

tokenized_dataset = {
    split: ds.map(tokenizer_fun, batched=True)
    for split, ds in formatted_dataset.items()
}

Map:   0%|          | 0/22450 [00:00<?, ? examples/s]

Map:   0%|          | 0/4182 [00:00<?, ? examples/s]

In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [15]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"], 
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

In [52]:
model = get_peft_model(model, lora_config)

In [17]:
import wandb

In [18]:
import os
os.environ["WANDB_API_KEY"] = "68855db0e94ebdfdc372c3e4569c7bb83f09d9f9"

In [88]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    weight_decay=0.2,
    num_train_epochs=2,
    per_device_train_batch_size=2 ,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8, 
    gradient_checkpointing=True,
    max_grad_norm=0.5,
    learning_rate=1e-5,
    lr_scheduler_type="cosine",
    warmup_ratio=0.02,
    no_cuda=False,
    local_rank=-1,
    fp16=True,  
    logging_steps=15,
    save_strategy="steps",
    save_steps=200,
    eval_strategy="steps",
    eval_steps=200,
    remove_unused_columns=False,
    dataloader_pin_memory=False,
    dataloader_num_workers=4,
    report_to= "wandb",
    run_name="TinyLlama-CUAD-3",
    load_best_model_at_end=True,  # Automatically load best checkpoint
    metric_for_best_model="eval_loss",  # Metric to monitor
    greater_is_better=False
)

In [20]:
from trl import SFTTrainer

In [21]:
from transformers import EarlyStoppingCallback

In [77]:
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    peft_config=lora_config,
    args=training_args,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=4)],
    
)

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [23]:
model.config.use_cache = False

In [24]:
trainer.train()

wandb: Currently logged in as: dheena731 (dheena731-jai-shriram-engineering-college) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Step,Training Loss,Validation Loss
200,1.631700,1.622266
400,1.387500,1.421554
600,1.318300,1.356585
800,1.244000,1.300283
1000,1.211000,1.250721
1200,1.133800,1.205421
1400,1.106200,1.175549
1600,1.061200,1.153918
1800,1.061900,1.139519
2000,1.046500,1.130394


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

TrainOutput(global_step=2808, training_loss=1.2032487703184795, metrics={'train_runtime': 16552.9917, 'train_samples_per_second': 2.713, 'train_steps_per_second': 0.17, 'total_flos': 7.14242786328576e+16, 'train_loss': 1.2032487703184795})

In [28]:
import evaluate


# Load tokenizer to decode predictions/labels
tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/results/checkpoint-2808")

# Load standard metrics
metric_em = evaluate.load("exact_match")
metric_f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    
    # Decode predictions and labels to text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in labels (ignored tokens) with tokenizer.pad_token_id before decoding
    labels = [[(l if l != -100 else tokenizer.pad_token_id) for l in label] for label in labels]
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Compute Exact Match and F1
    em_score = metric_em.compute(predictions=decoded_preds, references=decoded_labels)
    f1_score = metric_f1.compute(predictions=decoded_preds, references=decoded_labels)
    
    return {
        "exact_match": em_score["exact_match"],
        "f1": f1_score["f1"]
    }


In [65]:
model = AutoModelForCausalLM.from_pretrained(
    model_name)

In [66]:
model = PeftModel.from_pretrained(model, "//kaggle/working/results/checkpoint-2800")